# CS172 Assignment 1: CAPTCHAs
Shanghaitech University 2025 Fall Computer Vision I

Welcome to the first assignment of CS172. In this homework, we aim to build and train a neural network model to recognize CAPTCHAs consisting only of Arabic numerals (0-9). The primary focus is on generating synthetic verification code data, preprocessing it, and building a convolutional neural network (CNN) for character recognition.

## Part 0: Setup
Before running the notebook, ensure that your environment is properly set up. Install the necessary dependencies, which include `torch`, `torchvision`, `PIL`, and `matplotlib`. These can be installed via conda and pip using the command:
```
conda create -n 172a1 python=3.8
conda activate 172a1
pip install -r requirements.txt
```
Then you can run the following code to load the necessary packages.

In [ ]:
import torch, os
from cs172 import utils
from cs172 import generate
from cs172.datasets import ImageDataset
from cs172.networks import get_model
from cs172.pipeline import train, test

utils.set_seed(172)

## Part 1: Data Generation
Implement the `generate_verification` function in `cs172/generate.py`, where we generate synthetic images of verification codes. Each image contains a series of Arabic numerals. 
You can modify parameters such as colors, noise level and noise type.

After implementing this function, you can display some data by running the following code.

In [ ]:
need_rotate = True # Do not modify this line
images = [generate.generate_verification(need_rotate=need_rotate)[0] for _ in range(16)]
display(utils.merge_images(images))

To generate the training dataset, run the following code:

In [ ]:
need_rotate = True # if you do not want to rotate the image for training, feel free to set it to False
image_size = (260, 80)
data_num = 100000 # total data num
save_folder = 'data' # set data folder

# You can comment out the line to avoid duplicate data generation
generate.save_certification_data(image_size, data_num, save_folder, need_rotate)

Discussion:
- What are some real-world challenges you might encounter when working with CAPTCHA images compared to our synthetic ones (e.g., distortions, noise)?
    - Real-world captchas often feature strong distortions, curved baselines, complex background textures, close foreground/background colors, character adhesion, varying font sizes and shapes, motion blur, compression artifacts, and dynamic generation strategies. All of these make them much harder to model and segment than our controlled synthetic data.
- Would using different fonts or noise levels impact the model's learning ability? How would you test this?
    - Fonts and noise determine whether the model can learn generalized features. Their impact can be evaluated by generating datasets with different fonts/noise intensities, using the same training process to compare the performance on the validation set, or conducting grouped ablation experiments.

## Part 2: Data Preprossing
Implement `__len__` and `__getitem__` functions in `cs172/datasets.py` for a customized `Dataset` class.

Then you can run the following code to preprocess the generated images to fit the neural network's input and batch the data using Pytorch's `DataLoader`. 

You can change the `transform` variable which may include steps such as resizing the images, normalizing pixel values.

In [ ]:
# You can try different transform for data augmentation
# torchvision.transforms may be useful
transform = None

save_folder = 'data'
dataset = ImageDataset(imgdir=save_folder, transform=transform)

print(len(dataset))

ratio = 0.8
train_size = int(ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# you can modify batch_size if 'CUDA OUT OF MEMORY'
batch_size = 256
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True, num_workers=0)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size,
    shuffle=False, num_workers=0)

img, label = next(iter(train_dataloader))
print(img[0].min(), img[0].max())
print(label[0])
img.shape, label.shape

Discussion:
- How important is data normalization in neural networks? What could happen if you skip this step?
    - Normalization stabilizes feature distributions, balances gradients more effectively, and accelerates training. If normalization is skipped, gradients may explode or vanish, leading to slow convergence, extreme sensitivity to initialization, and even training failure.
- What effect does batch size have on model performance and training speed? How would you balance memory usage with batch size selection?
    - Small batches provide noisier gradients, which helps with generalization but leads to unstable training. Large batches, on the other hand, result in smoother convergence but require more memory and are prone to getting trapped in sharp minima. Typically, the largest possible batch size is chosen based on memory limits, followed by adjustments to balance stability and speed, often combined with learning rate scaling.

## Part 3: Model Architecture
Implement ResNet18 in `networks.py`, which contains `SimpleResBlock` and `ResNet18` class. 

For `SimpleResBlock`, you can follow this structure to implement each layer(weight layer, activation funcyion, residual...)

The residual block has two $3 \times 3$ convolutional layers with the same number of output channels. Each convolutional layer is followed by a batch normalization layer and a ReLU activation function. 
Then, we skip these two convolution operations and add the input directly before the final ReLU activation function. 
This kind of design requires that the output of the two convolutional layers has to be of the same shape as the input, so that they can be added together. 
If we want to change the number of channels, we need to introduce an additional $1 \times 1$ convolutional layer to transform the input into the desired shape for the addition operation. 

![ResBlock](assets/resnet-block.svg "picture is from https://d2l.ai/")

For `ResNet18`, you can follow this structure:

The first two layers of ResNet are the same as those of the GoogLeNet we described before: the $7 \times 7$ convolutional layer with 64 output channels and a stride of 2 is followed by the $3 \times 3$ max-pooling layer with a stride of 2. The difference is the batch normalization layer added after each convolutional layer in ResNet.

ResNet uses four modules made up of residual blocks, each of which uses several residual blocks with the same number of output channels. The number of channels in the first module is the same as the number of input channels. Since a max-pooling layer with a stride of 2 has already been used, it is not necessary to reduce the height and width. In the first residual block for each of the subsequent modules, the number of channels is doubled compared with that of the previous module, and the height and width are halved.

Then, we add all the modules to ResNet. Here, two residual blocks are used for each module. Lastly, we add a global average pooling layer, followed by the fully connected layer output.

There are four convolutional layers in each module (excluding the $1 \times 1$ convolutional layer). Together with the first $7 \times 7$ convolutional layer and the final fully connected layer, there are 18 layers in total. Therefore, this model is commonly known as ResNet-18. 

![CAPTCHAs](assets/resnet18-90.svg "picture is from https://d2l.ai/, which is a very useful deep learning book") 

Pictures and descriptions come from [Dive into Deep Learning](https://d2l.ai/), which is a very helpful deep learning book.

After implementing the networks architecture, you can run the following code to train the model.

In [ ]:
# Change model_name to compare with the ResNet implementation in torchvision
model_name = "myresnet18"
assert model_name in ("myresnet18", "resnet18", "resnet34")
model = get_model(model_name)
model

Discussion:
- What modifications could you make to the current architecture if the model struggles with overfitting or underfitting?
    - When underfitting occurs, you can increase the number of channels, add more residual blocks, switch to stronger activation functions, or expand the input resolution. For overfitting, you can introduce dropout, increase weight decay, apply data augmentation (such as rotation, affine transformation, and color perturbation), reduce the model width, or use label smoothing.
- Would you consider using a different type of model (like MLP) for this task? Why or why not?
    - MLPs flatten images, which leads to the loss of spatial structure, an enormous number of parameters, and poor translational invariance. Convolutional networks, with their local receptive fields and weight sharing, are more suitable for captcha recognition. Therefore, unless the input has been pre-extracted into feature vectors, it is not recommended to switch to MLPs.

## Part 4: Training
In`pipeline.py`, implement a training loop for the model. You will fine-tune hyperparameters like learning rate, batch size, and optimizer, while monitoring the model's performance with loss and accuracy metrics.

In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

if os.path.exists("model_weights.pth"):
    model_weights = torch.load("model_weights.pth")
    model.load_state_dict(model_weights)
else:
    # you can try different hyperparameters by passing args `lr`, `weight_decay`, `num_epoch`
    train(model, device, train_dataloader,lr=1e-4)

# save model weights, thus you can try different hyperparameters without retraining
torch.save(model.state_dict(), "model_weights.pth")

|   lr   | weight_decay | echo | train_image_accuracy | train_digit_accuracy | valid_image_accuracy | valid_digit_accuracy |
|--------|--------------|------|----------------------|----------------------|----------------------|----------------------|
| 1e-3   |     0.05     |  10  |       0.45996        |       0.01401        |       0.01232        |      0.00128         |
| 1e-3| 0.05|30|0.42848|0.01127|0.62000|0.02999|
|1e-4|0.05|10|0.99007|0.97012|0.99163|0.96415|
|5e-5|0.005|10|0.99931|0.99125|0.32312|0.00528|


First, noticing the low accuracy on the training set, I thought of increasing the number of training epochs, but the result was still unsatisfactory. Then I considered that the excessively large learning rate prevented effective gradient descent. So I reduced the learning rate and achieved good results. After that, I tried to find a smaller learning rate and weaken the regularization constraint, only to find that the performance on the validation set became worse. Finally, I concluded that this was caused by overfitting due to the excessively small learning rate and regularization.


Since the data in the test is divided into five parts, we will also split the training data into five equal parts to better align with the test.

Discussion:
- How would you approach adjusting the learning rate if the model's accuracy plateaus early? Could a learning rate schedule improve training?
    - If accuracy stagnates early, you can first try reducing the learning rate or switching to schedulers such as cosine annealing, StepLR, or OneCycle. Additionally, you can trigger ReduceLROnPlateau when a plateau is reached or use warm restarts to escape local optima.
- Would regularization methods (such as dropout or L2 regularization) be beneficial for this task? When would you apply them?
    - L2 regularization, dropout, data augmentation, label smoothing, and other methods can all alleviate overfitting. These regularization techniques should be introduced when the training accuracy is much higher than that of the validation set or when the validation loss rebounds
- How would you know when to stop training the model? Should early stopping be implemented, and if so, how would you determine when?
    - The decision to stop training should be based on the trend of loss/accuracy on the validation set. You can implement early stopping with a patience value (e.g., stopping training if the validation metric shows no improvement for N consecutive epochs) and save the best weights, thereby avoiding overfitting and ineffective training.

## Part 5: Evaluation
After training, evaluate the model's performance on the validation and test datasets using metrics such as digit accuracy and image accuracy. Visualize predictions and errors to understand the model’s strengths and weaknesses.
You could add more experiments and disscussion (like loss decay, metric table, hyperparameter ablation study) in this section.

In [ ]:
# test on your valid data
device = "cuda" if torch.cuda.is_available() else 'cpu'
metrics = test(model, device, valid_dataloader)
for key, value in metrics.items():
    print(f"{key}:", value.item())

Experiment with various hyperparameters — including learning rates, batch sizes, network architecture (e.g., number of layers, filter sizes, and activations), loss decay, and metric settings — on your own generated validation set. Analyze the results and report the best-performing configuration here:


In [ ]:
# test on sample test data
# There are 5 test sets in 'samples'. To receive full points for each test set,
# your model must achieve a digit accuracy ≥ 0.95 and an image accuracy ≥ 0.75.
folders = sorted(os.listdir('samples'))
for folder in folders:
    if folder == "test_alphabet":
        continue 

    test_dataset = ImageDataset(os.path.join('samples', folder), transform=transform)
    test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size,
        shuffle=False, num_workers=0,
    )
    metrics = test(model, device, test_dataloader)
    for key, value in metrics.items():
        print(f"{key} for {folder}:", value.item())
    
    model.eval()
    images, imgs, labels = zip(*[test_dataset.get_samples(idx) for idx in range(4)])
    labels = [label.reshape(5, 10).argmax(axis=-1).tolist() for label in labels]
    with torch.no_grad():
        preds = [model(torch.tensor(img).unsqueeze(0).to(device))[0].reshape(5, 10).argmax(axis=-1).cpu().numpy().tolist() for img in imgs]
    display(utils.merge_images(images, grid_size=(4, 1)))
    print(labels)
    print(preds)

Discussion:
- How reliable are accuracy metrics for evaluating the model? Would other metrics offer additional insights?
    - Single accuracy is generally reliable overall but may mask specific class biases or sequence errors. It is recommended to check per-character accuracy, precision/recall, confusion matrix, sequence edit distance, or character-level F1-score simultaneously to comprehensively evaluate the model.
- What kinds of errors are the most common—misrecognition of certain digits, complete failures? How could you address these types of errors?
    - Common errors include confusion between numbers with similar shapes (such as 1/7 and 3/8), missed judgments of entire captchas, and swapping of preceding and following characters. These issues can be improved by increasing the proportion of such hard samples in the dataset, applying color/geometric augmentation, using attention mechanisms or CTC decoding, and introducing language model constraints, among other methods.

## Part 6: Discussion
There are 11 discussion questions above. You need to answer all of them, each worth 1 point.

## Bonus: Alphabet CAPTCHA Recognition

In this part, you are encouraged to extend your model to recognize alphabetic CAPTCHAs (A–Z, a–z).
We have provided some pre-generated alphabet CAPTCHA images for test.

To complete this part, you should:
- Show several examples of the training data you generated or used.
- Report the modifications you made compared to the numeric CAPTCHA setting (e.g., data generation, model architecture, or training strategy).
- Report your training accuracy.
- Report your accuracy on the provided test set.

You can add new cells below to organize your experiments, show results, and summarize your findings.
Feel free to explore different ideas — creativity is encouraged!

In [ ]:
import os 
transform = None
batch_size = 128
folder = 'test_alphabet'
from cs172 import utils
from cs172 import generate_alpha
need_rotate = True 
images = [generate_alpha.generate_verification_alpha(need_rotate=need_rotate)[0] for _ in range(16)]
display(utils.merge_images(images))

In [ ]:
#---------------------------------------------------------
#
# Data generation for alphabet recognition
#
#---------------------------------------------------------



need_rotate = True # if you do not want to rotate the image for training, feel free to set it to False
image_size = (260, 80)
data_num = 100000 # total data num
save_folder = 'data_alpha' # set data folder

# You can comment out the line to avoid duplicate data generation
generate_alpha.save_certification_data_alpha(image_size, data_num, save_folder, need_rotate)

In [ ]:
import torch
from cs172.datasets_alpha import ImageDataset_alpha

In [ ]:
# You can try different transform for data augmentation
# torchvision.transforms may be useful
transform = None

save_folder = 'data_alpha'


from torchvision import transforms
from cs172.datasets_alpha import RandomGrayscaleOrColor

train_transform = transforms.Compose([
    transforms.Resize((227, 227)),
    RandomGrayscaleOrColor(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomAffine(
        degrees=25,
        translate=(0.15, 0.15),
        shear=10,
        scale=(0.85, 1.15),
        interpolation=transforms.InterpolationMode.BICUBIC
    ),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])
dataset = ImageDataset_alpha(imgdir=save_folder, transform=train_transform)
print(len(dataset))

ratio = 0.8
train_size = int(ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# you can modify batch_size if 'CUDA OUT OF MEMORY'
batch_size = 128
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True, num_workers=0)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size,
    shuffle=False, num_workers=0)

img, label = next(iter(train_dataloader))
print(img[0].min(), img[0].max())
print(label[0])
img.shape, label.shape

In [ ]:
from cs172.networks_alpha import get_model_alpha

model_name = "myresnet18"
assert model_name in ("myresnet18", "resnet18", "resnet34")
model = get_model_alpha(model_name)
model

In [ ]:

"""
------------------------------------------------

Test the networks or can be used as training demo

------------------------------------------------
"""
from cs172.pipeline_alpha import train_alpha, test_alpha

img, label = next(iter(train_dataloader))
print(img.shape, label.shape)   


from cs172.metrics_alpha import ImageAccuracy_alpha, DigitAccuracy_alpha
from torchmetrics import MetricCollection
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.to(device)

metrics = MetricCollection({
    "image_acc": ImageAccuracy_alpha(),
    "digit_acc": DigitAccuracy_alpha(),
}).to(device)

img, label = next(iter(train_dataloader))
img, label = img.to(device), label.to(device)
targets = label.argmax(dim=-1).long()   # [B,5]

for step in range(500):
    optimizer.zero_grad()
    pred = model(img).view(-1, 5, 52)
    loss = F.cross_entropy(pred.reshape(-1, 52), targets.reshape(-1))
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
        metrics.reset()
        metrics.update(pred, label)
        accs = metrics.compute()
        print(f"Step {step:03d} | Loss={loss.item():.4f} | "
              f"ImageAcc={accs['image_acc']:.3f} | DigitAcc={accs['digit_acc']:.3f}")



In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

if os.path.exists("model_weights_alpha.pth"):
    model_weights = torch.load("model_weights_alpha.pth")
    model.load_state_dict(model_weights)
else:
    # you can try different hyperparameters by passing args `lr`, `weight_decay`, `num_epoch`
    train_alpha(model, device, train_dataloader,lr=1e-4)

# save model weights, thus you can try different hyperparameters without retraining
torch.save(model.state_dict(), "model_weights_alpha.pth")

In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
metrics = test_alpha(model, device, valid_dataloader)
for key, value in metrics.items():
    print(f"{key}:", value.item())

In [ ]:
test_dataset = ImageDataset_alpha(os.path.join('samples', folder), transform=transform)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size,
    shuffle=False, num_workers=0,
)

In [ ]:
results = test_alpha(model, device, test_dataloader)
print("\n Test Results:")
for key, value in results.items():
    print(f"{key}: {value.item():.4f}")